In [10]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from xgboost.sklearn import XGBClassifier  #Additional scklearn functions
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [14]:
train = pd.read_csv('../../data/train.csv',engine='python')
target = 'target'
IDcol = 'ID_code'
print(df.head(3))

   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   

   var_196  var_197  var_198  var_199  
0   7.8784   8.5635  12.7803  -1.0914  
1   8.1267   8.7889  18.3560   1.9518  
2  -6.5213   8.2675  14.7222   0.3965  

[3 rows x 202 columns]


In [19]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['target'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['target'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['target'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [20]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

/usr/lib/python3/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report


NameError: name 'metrics' is not defined

In [ ]:
joblib.dump(xgb1, '../../models/modelXGB1.pkl', compress=9)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75,random_state=42)

Specify sufficient boosting iterations to reach a minimum

In [8]:
num_round = 1000

Leave most parameters as default

In [5]:
param = {'tree_method': 'gpu_hist', # Use GPU accelerated algorithm
         'objective': 'binary:logistic', # Specify multiclass classification       
         'eta'=0.1,
         'gamma'=0,
         'max_depth'=5,
         
         
         }

Convert input data from numpy to XGBoost format

In [6]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [7]:
accuracy = {} # Store accuracy result

In [9]:
modelXGB = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=accuracy)

[0]	test-merror:0.10404
[1]	test-merror:0.10292
[2]	test-merror:0.10212
[3]	test-merror:0.10212
[4]	test-merror:0.10198
[5]	test-merror:0.10198
[6]	test-merror:0.10198
[7]	test-merror:0.10194
[8]	test-merror:0.1018
[9]	test-merror:0.10146
[10]	test-merror:0.10138
[11]	test-merror:0.10122
[12]	test-merror:0.10108
[13]	test-merror:0.10058
[14]	test-merror:0.10034
[15]	test-merror:0.10008
[16]	test-merror:0.09992
[17]	test-merror:0.09966
[18]	test-merror:0.0994
[19]	test-merror:0.09918
[20]	test-merror:0.09898
[21]	test-merror:0.0989
[22]	test-merror:0.09836
[23]	test-merror:0.0983
[24]	test-merror:0.09796
[25]	test-merror:0.09794
[26]	test-merror:0.09776
[27]	test-merror:0.09756
[28]	test-merror:0.0974
[29]	test-merror:0.0969
[30]	test-merror:0.09684
[31]	test-merror:0.09648
[32]	test-merror:0.09632
[33]	test-merror:0.09606
[34]	test-merror:0.0959
[35]	test-merror:0.0959
[36]	test-merror:0.09558
[37]	test-merror:0.09544
[38]	test-merror:0.09522
[39]	test-merror:0.09494
[40]	test-merror:0

[322]	test-merror:0.08674
[323]	test-merror:0.08674
[324]	test-merror:0.08684
[325]	test-merror:0.08672
[326]	test-merror:0.08674
[327]	test-merror:0.0867
[328]	test-merror:0.08682
[329]	test-merror:0.08684
[330]	test-merror:0.08678
[331]	test-merror:0.08672
[332]	test-merror:0.08686
[333]	test-merror:0.08682
[334]	test-merror:0.0868
[335]	test-merror:0.0868
[336]	test-merror:0.08672
[337]	test-merror:0.08644
[338]	test-merror:0.08656
[339]	test-merror:0.08644
[340]	test-merror:0.08658
[341]	test-merror:0.08668
[342]	test-merror:0.08672
[343]	test-merror:0.0868
[344]	test-merror:0.08692
[345]	test-merror:0.08702
[346]	test-merror:0.08706
[347]	test-merror:0.08722
[348]	test-merror:0.08706
[349]	test-merror:0.087
[350]	test-merror:0.08694
[351]	test-merror:0.0869
[352]	test-merror:0.0867
[353]	test-merror:0.08672
[354]	test-merror:0.0867
[355]	test-merror:0.08668
[356]	test-merror:0.08672
[357]	test-merror:0.08686
[358]	test-merror:0.08672
[359]	test-merror:0.08692
[360]	test-merror:0.0

[641]	test-merror:0.08584
[642]	test-merror:0.08586
[643]	test-merror:0.08584
[644]	test-merror:0.08576
[645]	test-merror:0.08558
[646]	test-merror:0.08554
[647]	test-merror:0.0857
[648]	test-merror:0.08578
[649]	test-merror:0.08568
[650]	test-merror:0.0856
[651]	test-merror:0.0856
[652]	test-merror:0.08548
[653]	test-merror:0.08568
[654]	test-merror:0.08564
[655]	test-merror:0.08578
[656]	test-merror:0.08564
[657]	test-merror:0.08556
[658]	test-merror:0.08562
[659]	test-merror:0.08588
[660]	test-merror:0.08588
[661]	test-merror:0.08586
[662]	test-merror:0.08588
[663]	test-merror:0.08604
[664]	test-merror:0.08596
[665]	test-merror:0.0859
[666]	test-merror:0.08594
[667]	test-merror:0.08586
[668]	test-merror:0.08594
[669]	test-merror:0.08596
[670]	test-merror:0.08594
[671]	test-merror:0.0861
[672]	test-merror:0.086
[673]	test-merror:0.08614
[674]	test-merror:0.08622
[675]	test-merror:0.08638
[676]	test-merror:0.0864
[677]	test-merror:0.08652
[678]	test-merror:0.0864
[679]	test-merror:0.0

[960]	test-merror:0.0844
[961]	test-merror:0.0844
[962]	test-merror:0.08444
[963]	test-merror:0.08444
[964]	test-merror:0.08434
[965]	test-merror:0.08434
[966]	test-merror:0.08438
[967]	test-merror:0.08452
[968]	test-merror:0.08446
[969]	test-merror:0.0847
[970]	test-merror:0.08468
[971]	test-merror:0.08468
[972]	test-merror:0.08472
[973]	test-merror:0.08466
[974]	test-merror:0.08474
[975]	test-merror:0.08486
[976]	test-merror:0.0848
[977]	test-merror:0.08478
[978]	test-merror:0.08472
[979]	test-merror:0.08482
[980]	test-merror:0.08484
[981]	test-merror:0.08476
[982]	test-merror:0.0847
[983]	test-merror:0.08468
[984]	test-merror:0.08454
[985]	test-merror:0.08446
[986]	test-merror:0.08466
[987]	test-merror:0.0847
[988]	test-merror:0.0849
[989]	test-merror:0.08492
[990]	test-merror:0.08482
[991]	test-merror:0.08492
[992]	test-merror:0.08488
[993]	test-merror:0.08482
[994]	test-merror:0.0846
[995]	test-merror:0.0846
[996]	test-merror:0.0846
[997]	test-merror:0.08466
[998]	test-merror:0.08

In [11]:
joblib.dump(modelXGB, '../../models/modelXGB1000.pkl', compress=9)

['../../models/modelXGB1000.pkl']

In [12]:
modelXGB = joblib.load('../../models/modelXGB1000.pkl')

In [13]:
df = pd.read_csv('../../data/test.csv')
df = df.drop(columns=['ID_code'])

In [14]:
dtest = xgb.DMatrix(df)

In [15]:
ypred = modelXGB.predict( dtest )

In [16]:
np.savetxt("../../data/submit4.csv", ypred, delimiter=",")